<a href="https://colab.research.google.com/github/helonayala/sysid/blob/main/lagrange.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script demonstrates a symbolic workflow for modeling and linearizing a mechanical system.

The process involves:
* Deriving the nonlinear equations of motion (EOM) using the Lagrangian method.
* Linearizing the EOM around a chosen equilibrium point.

This modeling approach is fundamental for applications like system identification and model-based control design.

## EOMs to state space

The Lagrange equations are

$$\frac{d}{dt}\left\lbrack \frac{\partial L}{\partial {\dot{q} }_i }\right\rbrack -\frac{\partial L}{\partial q_i }=F_i^e ,i=1,2,\dots ,n$$

where $L(q,\dot{q} )=T(q,\dot{q} )-V(q)$ is the Lagrangian function, $q_i$ represents the $n$ degrees of freedom (generalized variables), and $F_i^e$ is the sum of the forcing and dissipative terms.

To obtain the equations of motion for mechanical systems, one simply performs the kinematics of the system using the variables $q_i ,{\dot{q} }_i$ and applies the Lagrange equation $n$ times. The result can be described in matrix form as

$$M(q)\ddot{q} +C(q,\dot{q} )+k(q)=\tau (u)$$

where $u$ is the vector of exogenous terms. The nonlinear state-space model

$$\dot{x} =f(x,u)$$

can then be obtained, considering

$$x = \begin{bmatrix} q \\ \dot{q} \end{bmatrix} $$

for a positive definite matrix $M(q)$, we have

$$\dot{x} = \begin{bmatrix} \dot{q} \\ M(q)^{-1}(\tau(u) - C(x) - k(x)) \end{bmatrix}$$

## Linearization

If the mechanical system has any nonlinearities, the resulting nonlinear state-space equations can be linearized around an equilibrium point using Taylor series expansion. For details, see the book by [Kluver, 2020](https://www.wiley.com/en-us/Dynamic+Systems%3A+Modeling%2C+Simulation%2C+and+Control%2C+2nd+Edition-p-9781119601869)

## Imports and functions

In [3]:
!pip install control
import sympy as sp
import numpy as np
import control as ct
import matplotlib.pyplot as plt
from IPython.display import display
from scipy.integrate import solve_ivp # Import the standard ODE solver

# Use SymPy's best available renderer for equations
sp.init_printing(use_latex='mathjax')

# --- Helper Functions for Generalization ---

def create_gen_vars(ndof, t):
    """Creates symbolic generalized coordinates, velocities, and accelerations."""
    q = sp.Matrix([sp.Function(f'q{i+1}')(t) for i in range(ndof)])
    qp = q.diff(t)
    qpp = qp.diff(t)
    return q, qp, qpp

def derive_eom(T, V, q, Fext, t):
    """Derives the symbolic EOM using the Euler-Lagrange equation."""
    ndof = len(q)
    L = T - V
    qp = q.diff(t)
    qpp = qp.diff(t)
    EOM_LHS = sp.Matrix([
        (sp.diff(sp.diff(L, qp[i]), t) - sp.diff(L, q[i])).simplify()
        for i in range(ndof)
    ])
    return sp.Eq(EOM_LHS, Fext), qpp


def get_state_space_representation(EOM, q, qp, qpp, u_vec):
    """Converts the symbolic EOM into a symbolic state-space representation."""
    ndof = len(q)
    sol = sp.solve(EOM, qpp, dict=True)[0]
    q_s = sp.symbols(f'q1:{ndof+1}_s')
    qp_s = sp.symbols(f'q1:{ndof+1}p_s')

    sub_map = {}
    for i in range(ndof):
        sub_map[q[i]] = q_s[i]
        sub_map[qp[i]] = qp_s[i]
        sub_map[sp.sin(q[i])] = sp.sin(q_s[i])
        sub_map[sp.cos(q[i])] = sp.cos(q_s[i])

    x = sp.Matrix([*q_s, *qp_s])
    u = u_vec

    xp_list = list(qp_s)
    for i in range(ndof):
        xp_list.append(sol[qpp[i]].subs(sub_map))
    xp = sp.Matrix(xp_list)

    return xp, x, u



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.6/549.6 kB 9.7 MB/s eta 0:00:00


## Energy terms

We use for this example the inverted pendulum on a cart. Only spung and unsprung masses are considered for inertia, and damping is present in $\theta$ angle of rotation (0 is pointing upwards).

In [8]:
# =============================================================================
# 1. System Definition
# =============================================================================
print("--- 1. System Definition ---")

# Define symbolic variables for the system parameters
m, M, L, g, t, b = sp.symbols('m M L g t b')

# Define symbolic variable for the input force
f_in = sp.symbols('f_in')

# Define generalized coordinates, velocities, and accelerations
ndof = 2
print(f"\nNumber of Degrees of Freedom (ndof): {ndof}")

q, qp, qpp = create_gen_vars(ndof, t)
print("\nGeneralized coordinates vector (q):")
display(q)
print("\nGeneralized velocities vector (qp):")
display(qp)
print("\nGeneralized accelerations vector (qpp):")
display(qpp)

# =============================================================================
# 2. Kinematics, Energy, and Forces for the Inverted Pendulum
# =============================================================================
print("\n--- 2. Kinematics, Energy, and Forces ---")

# Unpack coordinates and velocities for easier use
q1, q2 = q
q1p, q2p = qp

# Define kinematics (positions of components)
x_cart = q1
x_pend = q1 + L * sp.sin(q2)
y_pend = L * sp.cos(q2)

# Calculate velocities squared (for kinetic energy terms)
v_cart_sq = x_cart.diff(t)**2
v_pend_sq = x_pend.diff(t)**2 + y_pend.diff(t)**2

# Define potential energy (V) of the system
V = M * g * y_pend
print("\nPotential Energy (V):")
display(V)

# Define kinetic energy (T) of the system
T = 1/2 * m * v_cart_sq + 1/2 * M * sp.simplify(v_pend_sq)
print("\nKinetic Energy (T):")
display(T)

# Define generalized external forces, including friction on the pendulum joint
Fext = sp.Matrix([f_in, -b * q2p])
print("\nGeneralized external forces vector (Fext):")
display(Fext)

# Define the control input vector (for linearization)
u_vec = sp.Matrix([f_in])

--- 1. System Definition ---

Number of Degrees of Freedom (ndof): 2

Generalized coordinates vector (q):


⎡q₁(t)⎤
⎢     ⎥
⎣q₂(t)⎦


Generalized velocities vector (qp):


⎡d        ⎤
⎢──(q₁(t))⎥
⎢dt       ⎥
⎢         ⎥
⎢d        ⎥
⎢──(q₂(t))⎥
⎣dt       ⎦


Generalized accelerations vector (qpp):


...

L⋅M⋅g⋅cos(q₂(t))


Kinetic Energy (T):


      ⎛              2                                                   2⎞    ↪
      ⎜ 2 ⎛d        ⎞                   d         d           ⎛d        ⎞ ⎟    ↪
0.5⋅M⋅⎜L ⋅⎜──(q₂(t))⎟  + 2⋅L⋅cos(q₂(t))⋅──(q₁(t))⋅──(q₂(t)) + ⎜──(q₁(t))⎟ ⎟ +  ↪
      ⎝   ⎝dt       ⎠                   dt        dt          ⎝dt       ⎠ ⎠    ↪

↪                  2
↪       ⎛d        ⎞ 
↪ 0.5⋅m⋅⎜──(q₁(t))⎟ 
↪       ⎝dt       ⎠ 


Generalized external forces vector (Fext):


⎡    fᵢₙ     ⎤
⎢            ⎥
⎢   d        ⎥
⎢-b⋅──(q₂(t))⎥
⎣   dt       ⎦

## Obtaining state equations

In [5]:
# =============================================================================
# 3. Automated EOM Derivation
# =============================================================================
print("--- 3. Automated EOM Derivation ---")
EOM, qpp = derive_eom(T, V, q, Fext, t)

# Display the symbolic Equations of Motion
print("\nDerived Equations of Motion (EOM):")
display(EOM)

# =============================================================================
# 4. Automated State-Space Formulation
# =============================================================================
print("\n--- 4. Automated State-Space Formulation ---")
xp, x, u = get_state_space_representation(EOM, q, qp, qpp, u_vec)

# Display the symbolic state-space vectors
print("\nState Vector (x):")
display(x)
print("\nInput Vector (u):")
display(u)
print("\nState Derivative Vector (xp):")
display(xp)


--- 3. Automated EOM Derivation ---

Derived Equations of Motion (EOM):


⎡      ⎛                          2                 2            2        ⎞    ↪
⎢      ⎜               ⎛d        ⎞                 d            d         ⎟    ↪
⎢1.0⋅M⋅⎜- L⋅sin(q₂(t))⋅⎜──(q₂(t))⎟  + L⋅cos(q₂(t))⋅───(q₂(t)) + ───(q₁(t))⎟ +  ↪
⎢      ⎜               ⎝dt       ⎠                   2            2       ⎟    ↪
⎢      ⎝                                           dt           dt        ⎠    ↪
⎢                                                                              ↪
⎢                        ⎛   2                                      2        ⎞ ↪
⎢                        ⎜  d                                      d         ⎟ ↪
⎢                1.0⋅L⋅M⋅⎜L⋅───(q₂(t)) - g⋅sin(q₂(t)) + cos(q₂(t))⋅───(q₁(t))⎟ ↪
⎢                        ⎜    2                                      2       ⎟ ↪
⎣                        ⎝  dt                                     dt        ⎠ ↪

↪        2        ⎤                 
↪       d         ⎥                 
↪ 1.0⋅m⋅───(q₁(t))⎥               


--- 4. Automated State-Space Formulation ---

State Vector (x):


⎡q₁ ₛ⎤
⎢    ⎥
⎢q₂ ₛ⎥
⎢    ⎥
⎢q1pₛ⎥
⎢    ⎥
⎣q2pₛ⎦


Input Vector (u):


[fᵢₙ]


State Derivative Vector (xp):


⎡                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                                              ↪
⎢                                                          2       2           ↪
⎢                                                         L ⋅M⋅q2pₛ ⋅sin(q₂ ₛ) ↪
⎢                                                     ──────────────────────── ↪
⎢                                                              2               ↪
⎢                                                     - L⋅M⋅cos (q₂ ₛ) + L⋅M + ↪
⎢                                                                              ↪
⎢     2  2     2                                     2                         ↪
⎢    L ⋅M ⋅q2pₛ ⋅sin(q₂ ₛ)⋅cos(q₂ ₛ)              L⋅M ⋅g⋅sin(q₂ ₛ)             ↪
⎢- ─────────────────────────

In [6]:
# =============================================================================
# 5. System Linearization
# =============================================================================
print("--- 5. System Linearization ---")
# Define equilibrium point and numerical parameters
equilibrium_point = {s: 0 for s in x}
equilibrium_point.update({s: 0 for s in u})
param_values = {m: 0.05, M: 0.5, L: 10e-2, g: 9.81, b: 0.1}

# Linearize the system
A_sym = xp.jacobian(x)
B_sym = xp.jacobian(u)
A_lin_sym = A_sym.subs(equilibrium_point)
B_lin_sym = B_sym.subs(equilibrium_point)

Alin = np.array(A_lin_sym.evalf(subs=param_values, chop=True)).astype(float)
Blin = np.array(B_lin_sym.evalf(subs=param_values, chop=True)).astype(float)

# =============================================================================
# 6. System Simplification (if there are pole/zero cancelations)
# =============================================================================
print("\n--- 6. System Simplification ---")
# Define the system output (pendulum angle)
output_index = 1
Clin = np.zeros((1, 2 * ndof))
Clin[0, output_index] = 1.0
Dlin = np.array([[0]])

# Get the minimal realization to find the core 2nd-order dynamics.
sys_full_ss = ct.ss(Alin, Blin, Clin, Dlin)

print("\nMinimal System Transfer Function:")
sys_min_tf = ct.minreal(ct.tf(sys_full_ss))
print(sys_min_tf)

sys_min_ss = ct.ss(sys_min_tf)
A_min, B_min, C_min, D_min = sys_min_ss.A, sys_min_ss.B, sys_min_ss.C, sys_min_ss.D


--- 5. System Linearization ---

--- 6. System Simplification ---

Minimal System Transfer Function:
2 states have been removed from the model
<TransferFunction>: sys[2]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']


       -200
------------------
s^2 + 220 s - 1079

